## Pore size estimation with voxels 

### Algorithm 
* Import xyz file 
* Make cell lists 
* Make voxels and calculate particle collision list via cell list 
* For all voxels calculate if cell is occupied by more than half by particle via MC estimation 
  - if yes: voxcel element is set to occupied 
  - if no: voxcel element is set to empty 
* After voxcel occupation is calculated, calculated all connected voxcel clusters as pore. The pore size is given by the volume of all voxcels belonging to one pore 

### Next steps 

* draw voxcels as spheres and check for errors 
* parallelize loop 
* make python script for 2d and 3d systems 
* run test for 2D system 

* run code for all 

In [1]:
import numpy as np 
import pandas as pd 
import networkx as nx 
import matplotlib as pyplot 
from collections import defaultdict 

In [17]:
# Next to do dos:

# Test 
# get pore shape  

class Cells:
    def __init__(self,lmin,box_lx):
        self.nx = int(np.floor(box_lx/lmin))
        self.lx = box_lx/self.nx 
        
        # generate neigbhour basis 
        en = np.array([-1,0,1])
        enl = len(en)
        cx,cy,cz = np.meshgrid(en,en,en)
        ccx = np.reshape(cx,(1,enl*enl*enl))
        ccy = np.reshape(cy,(1,enl*enl*enl))
        ccz = np.reshape(cz,(1,enl*enl*enl))
        arr = np.transpose(np.concatenate((ccx,ccy,ccz), axis=0))
        print(len(arr),"Cell neighbours")
        self.neighbour_cell_basis = [ (i,j,k) for i,j,k in arr]
        print(self.neighbour_cell_basis)
        
        xx,yy,zz = np.meshgrid(range(self.nx),range(self.nx),range(self.nx))
        x = np.reshape(xx,(1,self.nx*self.nx*self.nx))
        y = np.reshape(yy,(1,self.nx*self.nx*self.nx))
        z = np.reshape(zz,(1,self.nx*self.nx*self.nx))
        arr = np.transpose(np.concatenate((x,y,z), axis=0))
        self.coords = [ (i,j,k) for i,j,k in arr]
        
        
    def list_generate(self, ppos, vcoords, vpos, origin):
        
        self.particle_list = defaultdict(list)
        for i,pos_i in enumerate(ppos):
            idx = np.floor((pos_i[0] - origin[0])/self.lx).astype(int)
            idy = np.floor((pos_i[1] - origin[1])/self.lx).astype(int)
            idz = np.floor((pos_i[2] - origin[2])/self.lx).astype(int)
            self.particle_list[(idx,idy,idz)].append(i)
        
        self.voxcel_list = defaultdict(list)
        for coord_i in vcoords:
            pos_i = vpos[coord_i]
            idx = np.floor((pos_i[0] - origin[0])/self.lx).astype(int)
            idy = np.floor((pos_i[1] - origin[1])/self.lx).astype(int)
            idz = np.floor((pos_i[2] - origin[2])/self.lx).astype(int)      
            self.voxcel_list[(idx,idy,idz)].append(coord_i)
        
    
    def get_neighbour_coords(self, coord_i):
         # get particles from neighbour cells   
        nncell = []
        for ncell in self.neighbour_cell_basis: 
            celli = np.array(coord_i) + np.array(ncell)
            celli = (celli[0]%self.nx, celli[1]%self.nx, celli[2]%self.nx)
            nncell.append(celli)
        
        return nncell 

class Box:
    def __init__(self,origin,lx):
        self.origin = origin
        self.lx = lx 
        self.center = origin + self.lx/2*np.ones(3)
    
class Spheres:
    sigma=1
    radius=sigma/2 
    def __init__(self, pos): 
        self.pos = pos
       
    def get_pos(self,id_i):
        return self.pos[id_i]
    
class Voxcels:
    def __init__(self,lx,vxn,origin):
        self.lx=lx
        self.outer_radius = lx*np.sqrt(3)/2
        self.inner_radius = lx/2   
        self.nx = vxn
        self.origin = origin
        self.volume = self.lx*self.lx*self.lx 
        
        
        en = np.array([-1,0,1])
        enl = len(en)
        cx,cy,cz = np.meshgrid(en,en,en)
        ccx = np.reshape(cx,(1,enl*enl*enl))
        ccy = np.reshape(cy,(1,enl*enl*enl))
        ccz = np.reshape(cz,(1,enl*enl*enl))
        arr = np.transpose(np.concatenate((ccx,ccy,ccz), axis=0))
        self.neighbour_cell_basis = [ (i,j,k) for i,j,k in arr]
        
        
         # initalize voxcel positions 
        xx,yy,zz = np.meshgrid(range(vxn),range(vxn), range(vxn))
        vx = np.reshape(xx,(1,self.nx*self.nx*self.nx))
        vy = np.reshape(yy,(1,self.nx*self.nx*self.nx))
        vz = np.reshape(zz,(1,self.nx*self.nx*self.nx))
        vxyz=np.transpose(np.concatenate((vx,vy,vz), axis=0))
        self.coords = [(i,j,k) for i,j,k in vxyz]
        
        self.pos = defaultdict(list)
        for ci in self.coords:
            self.pos[ci] = self.origin + self.lx*np.array(ci) + self.lx/2
        
        self.fill_state = defaultdict(list)
        for ci in self.coords: 
            self.fill_state[ci] = 0
    
    def get_vertices(self, coord_i):
        vpos = self.pos[coord_i]
        
        def get_edge_points(pos_i,ax_n,sign_p):
            vertex_n = np.zeros(3)
            vertex_n = pos_i + sign_p[0]*ax_n[:,0]/2. + sign_p[1]*ax_n[:,1]/2. + sign_p[2]*ax_n[:,2]/2.        
            return vertex_n

        ax_n = np.array([[1,0,0],[0,1,0],[0,0,1]])*self.lx
        vertices = np.zeros((8,3))
  
        vertices[0] = get_edge_points(vpos,ax_n,np.array([-1,-1,-1]))
        vertices[1] = get_edge_points(vpos,ax_n,np.array([+1,-1,-1]))
        vertices[2] = get_edge_points(vpos,ax_n,np.array([+1,+1,-1]))
        vertices[3] = get_edge_points(vpos,ax_n,np.array([-1,+1,-1]))
    
        vertices[4] = get_edge_points(vpos,ax_n,np.array([-1,-1,+1]))
        vertices[5] = get_edge_points(vpos,ax_n,np.array([+1,-1,+1]))
        vertices[6] = get_edge_points(vpos,ax_n,np.array([+1,+1,+1]))
        vertices[7] = get_edge_points(vpos,ax_n,np.array([-1,+1,+1]))

        return vertices
    
    def set_to_filled(self, coord_i):
        self.fill_state[coord_i] = 1 
        
    def set_to_empty(self, coord_i):
         self.voxcel_state[id_i] = 0 
         
    def get_neighbour_coords(self, coord_i):
        # get particles from neighbour cells   
        nncell = []
        for ncell in self.neighbour_cell_basis: 
            celli = np.array(coord_i) + np.array(ncell)
            celli = (celli[0]%self.nx, celli[1]%self.nx, celli[2]%self.nx)
            nncell.append(celli)
        
        return nncell 
  
    
def get_distance(v_pos,p_pos, blx):
    dist = p_pos - v_pos 
    dist = dist - blx*np.rint(dist/blx)
    return dist   


def get_vdistance(pos_v,pos_c,blx):
    
    m = len(pos_c)
    n = len(pos_v)
    
    pos_c_tiled = np.tile(pos_c,(n,1))
    pos_v_repeat = np.repeat(pos_v,m,axis=0)
    
    dist = pos_c_tiled - pos_v_repeat 
    dist = dist - blx*np.rint(dist/blx)
    ndist = np.linalg.norm(dist,axis=1)
    dist = np.reshape(dist,(m,n,3))
    ndist = np.reshape(ndist,(m,n))
    return dist, ndist 


def point_intersect(pdist, voxcels, coord_vi):
    
    intersect = 0
    vertices = voxcels.get_vertices(coord_vi)

    e01 = (vertices[1] - vertices[0])/voxcels.lx 
    e02 = (vertices[3] - vertices[0])/voxcels.lx 
    e03 = (vertices[4] - vertices[0])/voxcels.lx

    point_rhs = np.array([
        np.dot(pdist,e01),
        np.dot(pdist,e02),
        np.dot(pdist,e03)])
    
    length_point_rhs = np.linalg.norm(point_rhs)

    if length_point_rhs < 1:
        intersect=1

    return intersect 


def estimate_volume(voxcels, coord_vi, particles, coord_pi,blx):
    Ntrial = 100
    p_pos = np.reshape(particles.pos[coord_pi],(3,1))
    total_intersect=0
    points = voxcels.lx*(0.5 - np.random.sample((Ntrial,3)))
    points[:,0] =  points[:,0] + voxcels.pos[coord_vi][0]  
    points[:,1] =  points[:,1] + voxcels.pos[coord_vi][1] 
    points[:,2] =  points[:,2] + voxcels.pos[coord_vi][2]

    dist, ndist = get_vdistance(points,pos_c,blx) 
    total_intersect = len(ndist[ndist<particles.radius])
    
    #for ti in range(Ntrial):
    #    point = voxcels.pos[coord_vi] + voxcels.lx*(0.5 - np.random.sample(3))
    #    pdist=get_distance(point,p_pos,blx)
    #    if np.linalg.norm(pdist) < particles.radius:
    #        total_intersect+=1 
            
    return total_intersect/Ntrial


def get_overlap_volume(voxcels, coord_vi, particles, coord_pi, blx):
    v_pos = voxcels.pos[coord_vi]
    p_pos = particles.get_pos(coord_pi)
    overlap_volume_i = 0 
    # get distance 
    dist_vp = get_distance(v_pos,p_pos,blx)
    ldist_vp = np.linalg.norm(dist_vp)
                         
    # no overlap if distance larger than outer radius of voxcels + radius of sphere
    if ldist_vp > (voxcels.outer_radius + particles.radius):
        return overlap_volume_i 
                         
    # else, check inner radius of voxcel 
    else:
        # overlap, if distance smaller than inner radius:
        if ldist_vp < (voxcels.inner_radius + particles.radius):
            #print("overlap because dist smaller than inner radius")
            overlap_volume_i = estimate_volume(voxcels, coord_vi, particles, coord_pi, blx)
            #print("overlap volume MUST BE BIGGER THAN ZERO", overlap_volume_i)
            return overlap_volume_i 
                         
        # otherwise, detailed overlap check 
        else: 
            nearest_point_distance = (dist_vp/ldist_vp)*(1 - particles.radius)                    
            intersect = point_intersect(nearest_point_distance,voxcels, coord_vi)

            if intersect==0:
                return overlap_volume_i

            else:
                overlap_volume_i = estimate_volume(voxcels, coord_vi, particles, coord_pi, blx)
                return overlap_volume_i




In [ ]:
print("read data...")
fdir = "/Users/ada/Documents/Code_Development_2022/biogel/std_conditions_xyz/00115/poly/xyz-files"
fdata = "biogel_9900.0.xyz"

df_pos = pd.read_csv("{}/{}".format(fdir,fdata), delim_whitespace=True, names=["type,",'x','y','z'])
pos = df_pos[['x','y','z']].values 

print("initialize variables...")

# generate particles object 
particles = Spheres(pos)

# generate box object 
lx = 30 
origin = (-15)*np.ones(3)
box = Box(origin,lx)

# initalize cells object
# absolute lmin = particles.sigma 
lmin = particles.sigma 
cells = Cells(lmin,box.lx)

# initialize voxels object
voxcels_per_cell = 1
vxl = cells.lx/voxcels_per_cell
vxn = cells.nx*voxcels_per_cell 
voxcels = Voxcels(vxl,vxn, box.origin)

In [ ]:
print("generate cell lists...")
# initalize cells object 

cells.list_generate(particles.pos,voxcels.coords, voxcels.pos, box.origin)

In [ ]:
print("calculate voxcel state...")
for ci in cells.coords:
    print("CELLL", ci)
    overlap_candidates = []
    neighbours = cells.get_neighbour_coords(ci)
    for nci in neighbours:
        overlap_candidates.extend(cells.particle_list[nci])

    for coord_vi in cells.voxcel_list[ci]:
        #print("VOXCEL", coord_vi)
        overlap_volume = 0 
        i=0
        while(overlap_volume < 0.5 and (i!=len(overlap_candidates))):
            coord_pi = overlap_candidates[0]
            overlap_volume += get_overlap_volume(voxcels, coord_vi, particles, coord_pi, box.lx)
            i+=1 
            
        #print("overlap volume", overlap_volume)
        if overlap_volume>0.5:
            voxcels.set_to_filled(coord_vi)
        

In [ ]:
print("generate links between filled voxcels...")
voxcel_links = []    
# make linker list 
for coord_vi in voxcels.coords:
    if voxcel.state(coord_vi) == 0:
        for coord_ni in voxcel.get_neighbour_coords(coord_i, box):
            if voxcel.fill_state(coord_ni) == 0:
                voxel_links.append((coord_vi,coord_ni))


G = nx.Graph()
G.add_edges_from(voxcel_links)
domains = list(nx.connected_components(G))
domain_lengths = np.array([ len(domain) for domain in domains ])

print("calculate pore volume")
pore_volumes = voxels.volume*domain_lengths

print("pore volumes", pore_volumes)


## Version 2: Loop over particles first and check overlap with voxcels 

In [ ]:
print("read data...")
fdir = "/Users/ada/Documents/Code_Development_2022/biogel/std_conditions_xyz/00115/poly/xyz-files"
fdata = "biogel_9900.0.xyz"

df_pos = pd.read_csv("{}/{}".format(fdir,fdata), delim_whitespace=True, names=["type,",'x','y','z'])
pos = df_pos[['x','y','z']].values 

print("initialize variables...")

# generate particles object 
particles2 = Spheres(pos)

# generate box object 
lx = 30 
origin = (-15)*np.ones(3)
box2 = Box(origin,lx)

# initalize cells object
# absolute lmin = particles.sigma 
lmin = particles.sigma 
cells2 = Cells(lmin,box.lx)

# initialize voxels object
voxcels_per_cell = 4
vxl = cells.lx/voxcels_per_cell
vxn = cells.nx*voxcels_per_cell 
voxcels2 = Voxcels(vxl,vxn, box2.origin)

print("generate cell lists...")
# initalize cells object 
cells2.list_generate(particles2.pos,voxcels2.coords, voxcels2.pos, box2.origin)


print("calculate voxcel state...")
for ci in cells2.coords:
    print("CELLL", ci)
    overlap_candidates = []
    neighbours = cells2.get_neighbour_coords(ci)
    for nci in neighbours:
        overlap_candidates.extend(cells2.voxcel_list[nci])

    for coord_pi in cells2.particle_list[ci]:
        #print("VOXCEL", coord_vi)
        overlap_volume = 0 
        i=0
        for coord_vi in overlap_candidates: 
            overlap_volume = get_overlap_volume(voxcels2, coord_vi, particles2, coord_pi, box2.lx)
            #print("overlap volume", overlap_volume)
            if overlap_volume>0.2:
                voxcels2.set_to_filled(coord_vi)
                



### Version 3: Optimize by calculation distances in advance 

In [27]:
print("read data...")
fdir = "/Users/ada/Documents/Code_Development_2022/biogel/std_conditions_xyz/00115/poly/xyz-files"
fdata = "biogel_9900.0.xyz"

df_pos = pd.read_csv("{}/{}".format(fdir,fdata), delim_whitespace=True, names=["type,",'x','y','z'])
pos = df_pos[['x','y','z']].values 

print("initialize variables...")

# generate particles object 
particles = Spheres(pos)

# generate box object 
lx = 30 
origin = (-15)*np.ones(3)
box = Box(origin,lx)

# initalize cells object
# absolute lmin = particles.sigma 
lmin = particles.sigma 
cells = Cells(lmin,box.lx)

# initialize voxels object
voxcels_per_cell = 2
vxl = cells.lx/voxcels_per_cell
vxn = cells.nx*voxcels_per_cell 
voxcels = Voxcels(vxl,vxn, box.origin)
print(voxcels.get_neighbour_coords((0,0,0)))
print("generate cell lists...")
# initalize cells object 
cells.list_generate(particles.pos,voxcels.coords, voxcels.pos, box.origin)


print("calculate voxcel state...")
print(len(cells.coords))
for ci in cells.coords:
    #print(ci)
    # Overlap lists 
    vcoords = cells.voxcel_list[ci]
    #neighbours = cells.get_neighbour_coords(ci)
    #for nci in neighbours:
    #    vcoords.extend(cells.voxcel_list[nci])

    pcoords = []
    neighbours = cells.get_neighbour_coords(ci)
    for nci in neighbours:
        pcoords.extend(cells.particle_list[nci])
        
    pos_v = np.array([ voxcels.pos[i] for i in vcoords])
    pos_c = particles.pos[pcoords]
    dist, ndist = get_vdistance(pos_v,pos_c,box.lx)
    # overlap for sure 
    for i, j in np.argwhere(ndist<(particles.radius+voxcels.inner_radius)):
        overlap_volume_i = estimate_volume(voxcels, vcoords[j], particles, pcoords[i], box.lx)
        if overlap_volume_i>0.4:
            voxcels.set_to_filled(vcoords[j])
   


    # maybe overlap, need to check 
    #for i, j in np.argwhere((ndist>particles.radius) & (ndist<(particles.radius + voxcels.outer_radius))):
    #    nearest_point_distance = (dist[i,j]/ndist[i,j])*(1 - particles.radius)                    
    #    intersect = point_intersect(nearest_point_distance,voxcels,vcoords[j])

    #    if intersect==0:
    #        overlap_volume_i = 0

    #    else:
    #        overlap_volume_i = estimate_volume(voxcels, vcoords[j], particles, pcoords[i], box.lx)


read data...
initialize variables...
27 Cell neighbours
[(-1, -1, -1), (-1, -1, 0), (-1, -1, 1), (0, -1, -1), (0, -1, 0), (0, -1, 1), (1, -1, -1), (1, -1, 0), (1, -1, 1), (-1, 0, -1), (-1, 0, 0), (-1, 0, 1), (0, 0, -1), (0, 0, 0), (0, 0, 1), (1, 0, -1), (1, 0, 0), (1, 0, 1), (-1, 1, -1), (-1, 1, 0), (-1, 1, 1), (0, 1, -1), (0, 1, 0), (0, 1, 1), (1, 1, -1), (1, 1, 0), (1, 1, 1)]
[(59, 59, 59), (59, 59, 0), (59, 59, 1), (0, 59, 59), (0, 59, 0), (0, 59, 1), (1, 59, 59), (1, 59, 0), (1, 59, 1), (59, 0, 59), (59, 0, 0), (59, 0, 1), (0, 0, 59), (0, 0, 0), (0, 0, 1), (1, 0, 59), (1, 0, 0), (1, 0, 1), (59, 1, 59), (59, 1, 0), (59, 1, 1), (0, 1, 59), (0, 1, 0), (0, 1, 1), (1, 1, 59), (1, 1, 0), (1, 1, 1)]
generate cell lists...
calculate voxcel state...
27000


In [28]:
print("generate links between filled voxcels...")
voxcel_links = []    
# make linker list 
for coord_vi in voxcels.coords:
    if voxcels.fill_state[coord_vi] == 0:
        for coord_ni in voxcels.get_neighbour_coords(coord_vi):
            #print(coord_vi, coord_ni)
            #print(voxcels.fill_state[coord_vi], voxcels.fill_state[coord_ni])
            if (voxcels.fill_state[coord_ni] == 0) and (coord_vi != coord_ni):
                voxcel_links.append((coord_vi,coord_ni))

print("finished calculation of voxcel list")

G = nx.Graph()
G.add_edges_from(voxcel_links)
domains = list(nx.connected_components(G))
domain_lengths = np.array([ len(domain) for domain in domains ])

print("calculate pore volume")
pore_volumes = voxcels.volume*domain_lengths

print("pore volumes", pore_volumes)



generate links between filled voxcels...
finished calculation of voxcel list
calculate pore volume
pore volumes [20230.25]


In [33]:
domain_lengths

array([161842])

In [35]:
161842/(27000*8)

0.7492685185185185

### Version 4: Parallelized version 

In [ ]:
print("read data...")
fdir = "/Users/ada/Documents/Code_Development_2022/biogel/std_conditions_xyz/00115/poly/xyz-files"
fdata = "biogel_9900.0.xyz"

df_pos = pd.read_csv("{}/{}".format(fdir,fdata), delim_whitespace=True, names=["type,",'x','y','z'])
pos = df_pos[['x','y','z']].values 

print("initialize variables...")

# generate particles object 
particles = Spheres(pos)

# generate box object 
lx = 30 
origin = (-15)*np.ones(3)
box = Box(origin,lx)

# initalize cells object
# absolute lmin = particles.sigma 
lmin = 2*particles.sigma 
cells = Cells(lmin,box.lx)

# initialize voxels object
voxcels_per_cell = 4
vxl = cells.lx/voxcels_per_cell
vxn = cells.nx*voxcels_per_cell 
voxcels = Voxcels(vxl,vxn, box.origin)

print("generate cell lists...")
# initalize cells object 
cells.list_generate(particles.pos,voxcels.coords, voxcels.pos, box.origin)


def cell_loop(ci):
    # Overlap lists 
    vcoords = []
    neighbours = cells.get_neighbour_coords(ci)
    for nci in neighbours:
        vcoords.extend(cells.voxcel_list[nci])

    pcoords = []
    neighbours = cells.get_neighbour_coords(ci)
    for nci in neighbours:
        pcoords.extend(cells.particle_list[nci])
        
    pos_v = np.array([ voxcels.pos[i] for i in vcoords])
    pos_c = particles.pos[pcoords]
    dist, ndist = get_vdistance(pos_v,pos_c,box.lx)
    # overlap for sure 
    for i, j in np.argwhere(ndist<(particles.radius+voxcels.inner_radius)):
        overlap_volume_i = estimate_volume(voxcels, vcoords[j], particles, pcoords[i], box.lx)
        if overlap_volume_i>0.4:
            voxcels.set_to_filled(vcoords[j])


from joblib import Parallel, delayed
Parallel(n_jobs=6)(delayed(process)(ci) for ci in cells.coords)
            
            

### Write out the empty cells

In [ ]:
with open("pores.xyz",'w') as f:
    f.write("Particles of frame 0\n")
    N_nodes= 
    N_points = 8*N_nodes
    f.write("{}".format(N_points)
    for coord_vi in voxcels.coords:
        if voxcel.state(coord_vi) == 0:
            vertices = voxcels.get_vertices(coord_vi)
        
        